In [ ]:
# import resource


# def limit_memory(maxsize):
#     soft, hard = resource.getrlimit(resource.RLIMIT_AS)
#     # MEMORY LIMIT IN BYTES
#     resource.setrlimit(resource.RLIMIT_AS, (maxsize, hard))


# # IN GIGABYTE
# limit_memory(30 * 1024 * 1024 * 1024)

In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline

import gzip
from functools import reduce
from pathlib import Path
from zipfile import ZipFile

import polars as pl
import pandas as pd
import pyarrow as pa
from pandas import DataFrame
from pyarrow import parquet
from tqdm.autonotebook import tqdm

from tsdm.datasets import MIMIC_IV

In [ ]:
# ds = MIMIC_IV(initialize=False)
# ds.rawdata_paths

DATASET_PATH = Path("custom_processed")
RAWDATA_PATH = Path("mimic-iv-2.2.zip")

In [ ]:
filelist = [
    "mimic-iv-2.2/CHANGELOG.txt",
    "mimic-iv-2.2/LICENSE.txt",
    "mimic-iv-2.2/SHA256SUMS.txt",
    "mimic-iv-2.2/hosp/admissions.csv.gz",
    "mimic-iv-2.2/hosp/d_hcpcs.csv.gz",
    "mimic-iv-2.2/hosp/d_icd_diagnoses.csv.gz",
    "mimic-iv-2.2/hosp/d_icd_procedures.csv.gz",
    "mimic-iv-2.2/hosp/d_labitems.csv.gz",
    "mimic-iv-2.2/hosp/diagnoses_icd.csv.gz",
    "mimic-iv-2.2/hosp/drgcodes.csv.gz",
    "mimic-iv-2.2/hosp/emar.csv.gz",
    "mimic-iv-2.2/hosp/emar_detail.csv.gz",
    "mimic-iv-2.2/hosp/hcpcsevents.csv.gz",
    "mimic-iv-2.2/hosp/labevents.csv.gz",
    "mimic-iv-2.2/hosp/microbiologyevents.csv.gz",
    "mimic-iv-2.2/hosp/omr.csv.gz",
    "mimic-iv-2.2/hosp/patients.csv.gz",
    "mimic-iv-2.2/hosp/pharmacy.csv.gz",
    "mimic-iv-2.2/hosp/poe.csv.gz",
    "mimic-iv-2.2/hosp/poe_detail.csv.gz",
    "mimic-iv-2.2/hosp/prescriptions.csv.gz",
    "mimic-iv-2.2/hosp/procedures_icd.csv.gz",
    "mimic-iv-2.2/hosp/provider.csv.gz",
    "mimic-iv-2.2/hosp/services.csv.gz",
    "mimic-iv-2.2/hosp/transfers.csv.gz",
    "mimic-iv-2.2/icu/caregiver.csv.gz",
    "mimic-iv-2.2/icu/chartevents.csv.gz",
    "mimic-iv-2.2/icu/d_items.csv.gz",
    "mimic-iv-2.2/icu/datetimeevents.csv.gz",
    "mimic-iv-2.2/icu/icustays.csv.gz",
    "mimic-iv-2.2/icu/ingredientevents.csv.gz",
    "mimic-iv-2.2/icu/inputevents.csv.gz",
    "mimic-iv-2.2/icu/outputevents.csv.gz",
    "mimic-iv-2.2/icu/procedureevents.csv.gz",
]

In [ ]:
with ZipFile(RAWDATA_PATH, "r") as file:
    a = set(file.namelist())
    b = set(filelist)
    if not a == b:
        raise ValueError(f"The dataset contains unknown files {a-b}!")


def load_gz_table(archive, fname, **options):
    with archive.open(str(fname), "r") as compressed_file:
        with gzip.open(compressed_file, "r") as file:
            # table = pd.read_csv(file)
            # return table
            table = pa.csv.read_csv(file, **options)
            return table


def load_txt(archive, fname):
    with archive.open(str(fname), "r") as file:
        return str(file.read(), encoding="utf8")


def compute_entropy(value_counts) -> float:
    counts = pa.compute.struct_field(value_counts, 1)
    n = len(counts)
    freqs = pa.compute.divide(
        pa.compute.cast(counts, pa.float64()),
        pa.compute.sum(counts),
    )

    H = pa.compute.divide(
        pa.compute.sum(pa.compute.multiply(freqs, pa.compute.log2(freqs))),
        pa.compute.log2(n),
    )
    return -H.as_py()


def get_table(fname, **options) -> DataFrame:
    fname = Path(fname)
    with ZipFile(RAWDATA_PATH, "r") as archive:
        match fname.suffix:
            case ".gz":
                return load_gz_table(archive, fname, **options)
            case ".txt":
                return load_txt(archive, fname, **options)
            case _:
                raise ValueError


def table_info(table) -> None:
    size = table.nbytes / (1024 * 1024 * 1024)
    print(f"shape={table.shape}  {size=:.3f} GiB")
    max_key = max(map(len, table.column_names))
    for name, col in tqdm(zip(table.column_names, table.columns)):
        num_total = len(col)
        num_null = pa.compute.sum(pa.compute.is_null(col)).as_py()
        value_counts = col.value_counts()
        num_uniques = len(value_counts) - bool(num_null)
        nulls = f"{num_null / num_total:7.2%}" if num_null else f"{'None':7s}"
        uniques = (
            num_uniques / (num_total - num_null)
            if num_total > num_null
            else num_uniques / num_total
        )
        entropy = compute_entropy(value_counts)
        dtype = str(col.type)[:10]
        print(
            f"{name:{max_key}s}  {nulls=:s}  {num_uniques=:9d} ({uniques:7.2%})  {entropy=:7.2%}  {dtype=:s}"
        )


def filter_nulls(table: pa.Table, cols: list[str], aggregation="or") -> pa.Table:
    def or_(variables):
        n = len(variables)
        if n == 0:
            return False
        if n == 1:
            return variables[0]
        return pa.compute.or_(
            or_(variables[: n // 2]),
            or_(variables[n // 2 :]),
        )

    def and_(variables):
        n = len(variables)
        if n == 0:
            return True
        if n == 1:
            return variables[0]
        return pa.compute.and_(
            or_(variables[: n // 2]),
            or_(variables[n // 2 :]),
        )

    masks = [table[col].is_null() for col in cols]
    mask = pa.compute.invert(or_(masks))
    return table.filter(mask)


with ZipFile(RAWDATA_PATH, "r") as file:
    if not file.namelist() == filelist:
        raise ValueError("The dataset contains unknown files!")
    print(dir(file))

    filesizes = {info.filename: info.file_size for info in file.infolist()}
    # print([x.filename for x in file.infolist()])

pd.Series({key: filesizes[key] for key in filesizes})

In [ ]:
ID_TYPE = "uint32"
VALUE_TYPE = "float32"
TIME_TYPE = "timestamp[s]"
DATE_TYPE = "date32[day]"
BOOL_TYPE = "bool"
STRING_TYPE = "string"
DICT_TYPE = pa.dictionary("int32", "string")


NULL_VALUES = [
    "",
    " ",
    "  ",
    "   ",
    "    ",
    "     ",
    "      ",
    "       ",
    "        ",
    "-",
    "---",
    "----",
    "-----",
    "-------",
    "?",
    "UNABLE TO OBTAIN",
    "UNKNOWN",
    "Unknown",
    "unknown",
    ".",
    ".*.",
    "___.",
    "_",
    "__",
    "___",
]
TRUE_VALUES = ["Y", "Yes", "1"]
FALSE_VALUES = ["N", "No", "0"]


column_types = dict(
    chartevents={
        "subject_id": ID_TYPE,
        "hadm_id": ID_TYPE,
        "stay_id": ID_TYPE,
        "caregiver_id": ID_TYPE,
        "itemid": ID_TYPE,
        "charttime": TIME_TYPE,
        "storetime": TIME_TYPE,
        "value": STRING_TYPE,  # string → float
        "valuenum": VALUE_TYPE,
        "valueuom": DICT_TYPE,
        "warning": BOOL_TYPE,
    },
    inputevents={
        "subject_id": ID_TYPE,
        "hadm_id": ID_TYPE,
        "stay_id": ID_TYPE,
        "caregiver_id": ID_TYPE,
        "starttime": TIME_TYPE,
        "endtime": TIME_TYPE,
        "storetime": TIME_TYPE,
        "itemid": ID_TYPE,
        "amount": VALUE_TYPE,
        "amountuom": DICT_TYPE,
        "rate": VALUE_TYPE,
        "rateuom": DICT_TYPE,
        "orderid": ID_TYPE,
        "linkorderid": ID_TYPE,
        "ordercategoryname": DICT_TYPE,
        "secondaryordercategoryname": DICT_TYPE,
        "ordercomponenttypedescription": DICT_TYPE,
        "ordercategorydescription": DICT_TYPE,
        "patientweight": VALUE_TYPE,
        "totalamount": VALUE_TYPE,
        "totalamountuom": DICT_TYPE,
        "isopenbag": BOOL_TYPE,
        "continueinnextdept": BOOL_TYPE,
        "statusdescription": DICT_TYPE,
        "originalamount": VALUE_TYPE,
        "originalrate": VALUE_TYPE,
    },
    outputevents={
        "subject_id": ID_TYPE,
        "hadm_id": ID_TYPE,
        "stay_id": ID_TYPE,
        "caregiver_id": ID_TYPE,
        "charttime": TIME_TYPE,
        "storetime": TIME_TYPE,
        "itemid": ID_TYPE,
        "value": VALUE_TYPE,
        "valueuom": DICT_TYPE,
    },
    procedureevents={
        "subject_id": ID_TYPE,
        "hadm_id": ID_TYPE,
        "stay_id": ID_TYPE,
        "caregiver_id": ID_TYPE,
        "starttime": TIME_TYPE,
        "endtime": TIME_TYPE,
        # "storetime": TIME_TYPE,
        "itemid": ID_TYPE,
        "value": VALUE_TYPE,
        "valueuom": DICT_TYPE,
        "location": DICT_TYPE,
        "locationcategory": DICT_TYPE,
        "orderid": ID_TYPE,
        "linkorderid": ID_TYPE,
        "ordercategoryname": DICT_TYPE,
        "ordercategorydescription": DICT_TYPE,
        "patientweight": VALUE_TYPE,
        "isopenbag": BOOL_TYPE,
        "continueinnextdept": BOOL_TYPE,
        "statusdescription": DICT_TYPE,
        "originalamount": VALUE_TYPE,
        "originalrate": BOOL_TYPE,
    },
    datetimeevents={
        "subject_id": ID_TYPE,
        "hadm_id": ID_TYPE,
        "stay_id": ID_TYPE,
        "caregiver_id": ID_TYPE,
        "charttime": TIME_TYPE,
        "storetime": TIME_TYPE,
        "itemid": ID_TYPE,
        "value": TIME_TYPE,
        "valueuom": DICT_TYPE,
        "warning": BOOL_TYPE,
    },
    ingredientevents={
        "subject_id": ID_TYPE,
        "hadm_id": ID_TYPE,
        "stay_id": ID_TYPE,
        "caregiver_id": ID_TYPE,
        "starttime": TIME_TYPE,
        "endtime": TIME_TYPE,
        "storetime": TIME_TYPE,
        "itemid": ID_TYPE,
        "amount": VALUE_TYPE,
        "amountuom": DICT_TYPE,
        "rate": VALUE_TYPE,
        "rateuom": DICT_TYPE,
        "orderid": ID_TYPE,
        "linkorderid": ID_TYPE,
        "statusdescription": DICT_TYPE,
        "originalamount": VALUE_TYPE,
        "originalrate": VALUE_TYPE,
    },
    icustays={
        "subject_id": ID_TYPE,
        "hadm_id": ID_TYPE,
        "stay_id": ID_TYPE,
        "first_careunit": DICT_TYPE,
        "last_careunit": DICT_TYPE,
        "intime": TIME_TYPE,
        "outtime": TIME_TYPE,
        "los": VALUE_TYPE,
    },
    d_items={
        "itemid": ID_TYPE,
        "label": STRING_TYPE,
        "abbreviation": STRING_TYPE,
        "linksto": DICT_TYPE,
        "category": DICT_TYPE,
        "unitname": DICT_TYPE,
        "param_type": DICT_TYPE,
        "lownormalvalue": VALUE_TYPE,
        "highnormalvalue": VALUE_TYPE,
    },
    caregiver={
        "caregiver_id": ID_TYPE,
    },
);

In [ ]:
column_types = dict(
    transfers={
        "subject_id": ID_TYPE,
        "hadm_id": ID_TYPE,
        "transfer_id": ID_TYPE,
        "eventtype": DICT_TYPE,
        "careunit": DICT_TYPE,
        "intime": TIME_TYPE,
        "outtime": TIME_TYPE,
    },
    admissions={
        "subject_id": ID_TYPE,
        "hadm_id": ID_TYPE,
        "admittime": TIME_TYPE,
        "dischtime": TIME_TYPE,
        "deathtime": TIME_TYPE,
        "admission_type": DICT_TYPE,
        "admit_provider_id": DICT_TYPE,
        "admission_location": DICT_TYPE,
        "discharge_location": DICT_TYPE,
        "insurance": DICT_TYPE,
        "language": DICT_TYPE,
        "marital_status": DICT_TYPE,
        "race": DICT_TYPE,
        "edregtime": TIME_TYPE,
        "edouttime": TIME_TYPE,
        "hospital_expire_flag": BOOL_TYPE,
    },
    services={
        "subject_id": ID_TYPE,
        "hadm_id": ID_TYPE,
        "transfertime": TIME_TYPE,
        "prev_service": DICT_TYPE,
        "curr_service": DICT_TYPE,
    },
    provider={
        "provider_id": STRING_TYPE,
    },
    procedures_icd={
        "subject_id": ID_TYPE,
        "hadm_id": ID_TYPE,
        "seq_num": ID_TYPE,
        "chartdate": DATE_TYPE,
        "icd_code": DICT_TYPE,
        "icd_version": ID_TYPE,
    },
    prescriptions={
        "subject_id": ID_TYPE,
        "hadm_id": ID_TYPE,
        "pharmacy_id": ID_TYPE,
        "poe_id": STRING_TYPE,
        "poe_seq": ID_TYPE,
        "order_provider_id": DICT_TYPE,
        "starttime": TIME_TYPE,
        "stoptime": TIME_TYPE,
        "drug_type": DICT_TYPE,
        "drug": DICT_TYPE,
        "formulary_drug_cd": DICT_TYPE,
        "gsn": DICT_TYPE,
        "ndc": DICT_TYPE,
        "prod_strength": DICT_TYPE,
        "form_rx": DICT_TYPE,
        "dose_val_rx": STRING_TYPE,  # string → float
        "dose_unit_rx": DICT_TYPE,
        "form_val_disp": STRING_TYPE,  # string → float
        "form_unit_disp": DICT_TYPE,
        "doses_per_24_hrs": VALUE_TYPE,
        "route": DICT_TYPE,
    },
    poe_detail={
        "poe_id": STRING_TYPE,
        "poe_seq": ID_TYPE,
        "subject_id": ID_TYPE,
        "field_name": DICT_TYPE,
        "field_value": STRING_TYPE,  # unstack column
    },
    poe={
        "poe_id": STRING_TYPE,
        "poe_seq": ID_TYPE,
        "subject_id": ID_TYPE,
        "hadm_id": ID_TYPE,
        "ordertime": TIME_TYPE,
        "order_type": DICT_TYPE,
        "order_subtype": DICT_TYPE,
        "transaction_type": DICT_TYPE,
        "discontinue_of_poe_id": STRING_TYPE,
        "discontinued_by_poe_id": STRING_TYPE,
        "order_provider_id": DICT_TYPE,
        "order_status": DICT_TYPE,
    },
    pharmacy={
        "subject_id": ID_TYPE,
        "hadm_id": ID_TYPE,
        "pharmacy_id": ID_TYPE,
        "poe_id": STRING_TYPE,
        "starttime": TIME_TYPE,
        "stoptime": TIME_TYPE,
        "medication": DICT_TYPE,
        "proc_type": DICT_TYPE,
        "status": DICT_TYPE,
        "entertime": TIME_TYPE,
        "verifiedtime": TIME_TYPE,
        "route": DICT_TYPE,
        "frequency": DICT_TYPE,
        "disp_sched": DICT_TYPE,
        "infusion_type": DICT_TYPE,
        "sliding_scale": BOOL_TYPE,
        "lockout_interval": DICT_TYPE,
        "basal_rate": VALUE_TYPE,
        "one_hr_max": DICT_TYPE,
        "doses_per_24_hrs": VALUE_TYPE,
        "duration": VALUE_TYPE,
        "duration_interval": DICT_TYPE,
        "expiration_value": VALUE_TYPE,
        "expiration_unit": DICT_TYPE,
        "expirationdate": TIME_TYPE,
        "dispensation": DICT_TYPE,
        "fill_quantity": DICT_TYPE,
    },
    patients={
        "subject_id": ID_TYPE,
        "gender": DICT_TYPE,
        "anchor_age": ID_TYPE,
        "anchor_year": ID_TYPE,
        "anchor_year_group": DICT_TYPE,
        "dod": DATE_TYPE,
    },
    omr={
        "subject_id": ID_TYPE,
        "chartdate": DATE_TYPE,
        "seq_num": ID_TYPE,
        "result_name": STRING_TYPE,  # unstack
        "result_value": STRING_TYPE,  # unstack
        # split blood pressure into 2 floats (blood pressure systolic/diastolic).
    },
    microbiologyevents={
        "microevent_id": ID_TYPE,
        "subject_id": ID_TYPE,
        "hadm_id": ID_TYPE,
        "micro_specimen_id": ID_TYPE,
        "order_provider_id": DICT_TYPE,
        "chartdate": TIME_TYPE,
        "charttime": TIME_TYPE,
        "spec_itemid": ID_TYPE,
        "spec_type_desc": DICT_TYPE,
        "test_seq": ID_TYPE,
        "storedate": TIME_TYPE,
        "storetime": TIME_TYPE,
        "test_itemid": ID_TYPE,
        "test_name": DICT_TYPE,
        "org_itemid": ID_TYPE,
        "org_name": DICT_TYPE,
        "isolate_num": ID_TYPE,
        "quantity": DICT_TYPE,
        "ab_itemid": ID_TYPE,
        "ab_name": DICT_TYPE,
        "dilution_text": STRING_TYPE,  # convert to float
        "dilution_comparison": DICT_TYPE,
        "dilution_value": VALUE_TYPE,
        "interpretation": DICT_TYPE,
        "comments": STRING_TYPE,
    },
    labevents={
        "labevent_id": ID_TYPE,
        "subject_id": ID_TYPE,
        "hadm_id": ID_TYPE,
        "specimen_id": ID_TYPE,
        "itemid": ID_TYPE,
        "order_provider_id": DICT_TYPE,
        "charttime": TIME_TYPE,
        "storetime": TIME_TYPE,
        "value": STRING_TYPE,  # cast Float32
        "valuenum": VALUE_TYPE,
        "valueuom": DICT_TYPE,
        "ref_range_lower": VALUE_TYPE,
        "ref_range_upper": VALUE_TYPE,
        "flag": DICT_TYPE,
        "priority": DICT_TYPE,
        "comments": STRING_TYPE,
    },
    hcpcsevents={
        "subject_id": ID_TYPE,
        "hadm_id": ID_TYPE,
        "chartdate": DATE_TYPE,
        "hcpcs_cd": DICT_TYPE,
        "seq_num": ID_TYPE,
        "short_description": DICT_TYPE,
    },
    emar_detail={
        "subject_id": ID_TYPE,
        "emar_id": STRING_TYPE,
        "emar_seq": ID_TYPE,
        "parent_field_ordinal": DICT_TYPE,
        "administration_type": DICT_TYPE,
        "pharmacy_id": ID_TYPE,
        "barcode_type": DICT_TYPE,
        "reason_for_no_barcode": DICT_TYPE,
        "complete_dose_not_given": BOOL_TYPE,
        "dose_due": STRING_TYPE,  # cast float
        "dose_due_unit": DICT_TYPE,
        "dose_given": STRING_TYPE,  # cast float
        "dose_given_unit": DICT_TYPE,
        "will_remainder_of_dose_be_given": BOOL_TYPE,
        "product_amount_given": STRING_TYPE,  # cast float
        "product_unit": DICT_TYPE,
        "product_code": DICT_TYPE,
        "product_description": DICT_TYPE,
        "product_description_other": DICT_TYPE,
        "prior_infusion_rate": STRING_TYPE,  # cast float
        "infusion_rate": STRING_TYPE,  # cast float
        "infusion_rate_adjustment": DICT_TYPE,
        "infusion_rate_adjustment_amount": STRING_TYPE,  # cast float
        "infusion_rate_unit": DICT_TYPE,
        "route": DICT_TYPE,
        "infusion_complete": BOOL_TYPE,
        "completion_interval": DICT_TYPE,
        "new_iv_bag_hung": BOOL_TYPE,
        "continued_infusion_in_other_location": BOOL_TYPE,
        "restart_interval": DICT_TYPE,
        "side": DICT_TYPE,
        "site": DICT_TYPE,
        "non_formulary_visual_verification": BOOL_TYPE,
    },
    emar={
        "subject_id": ID_TYPE,
        "hadm_id": ID_TYPE,
        "emar_id": STRING_TYPE,
        "emar_seq": ID_TYPE,
        "poe_id": STRING_TYPE,
        "pharmacy_id": ID_TYPE,
        "enter_provider_id": DICT_TYPE,
        "charttime": TIME_TYPE,
        "medication": DICT_TYPE,
        "event_txt": DICT_TYPE,
        "scheduletime": TIME_TYPE,
        "storetime": TIME_TYPE,
    },
    drgcodes={
        "subject_id": ID_TYPE,
        "hadm_id": ID_TYPE,
        "drg_type": DICT_TYPE,
        "drg_code": ID_TYPE,
        "description": DICT_TYPE,
        "drg_severity": ID_TYPE,
        "drg_mortality": ID_TYPE,
    },
    diagnoses_icd={
        "subject_id": ID_TYPE,
        "hadm_id": ID_TYPE,
        "seq_num": ID_TYPE,
        "icd_code": DICT_TYPE,
        "icd_version": ID_TYPE,
    },
    d_labitems={
        "itemid": ID_TYPE,
        "label": STRING_TYPE,
        "fluid": DICT_TYPE,
        "category": DICT_TYPE,
    },
    d_icd_procedures={
        "icd_code": STRING_TYPE,
        "icd_version": ID_TYPE,
        "long_title": STRING_TYPE,
    },
    d_icd_diagnoses={
        "icd_code": STRING_TYPE,
        "icd_version": ID_TYPE,
        "long_title": STRING_TYPE,
    },
    d_hcpcs={
        "code": STRING_TYPE,
        "category": ID_TYPE,
        "long_description": STRING_TYPE,
        "short_description": "dictionary",
    },
);

## d_hcpcs

In [ ]:
table = get_table(
    "mimic-iv-2.2/hosp/d_hcpcs.csv.gz",
    convert_options=pa.csv.ConvertOptions(
        column_types=column_types["d_hcpcs"],
        strings_can_be_null=True,
        null_values=NULL_VALUES,
        true_values=TRUE_VALUES,
        false_values=FALSE_VALUES,
    ),
)
table_info(table)
# pa.compute.unique(table["poe_seq"])

## d_icd_diagnoses

In [ ]:
table = get_table(
    "mimic-iv-2.2/hosp/d_icd_diagnoses.csv.gz",
    convert_options=pa.csv.ConvertOptions(
        column_types=column_types["d_icd_diagnoses"],
        strings_can_be_null=True,
        null_values=NULL_VALUES,
        true_values=TRUE_VALUES,
        false_values=FALSE_VALUES,
    ),
)
table_info(table)
# pa.compute.unique(table["poe_seq"])

## d_icd_procedures

In [ ]:
table = get_table(
    "mimic-iv-2.2/hosp/d_icd_procedures.csv.gz",
    convert_options=pa.csv.ConvertOptions(
        # column_types=column_types["d_icd_procedures"],
        strings_can_be_null=True,
        null_values=NULL_VALUES,
        true_values=TRUE_VALUES,
        false_values=FALSE_VALUES,
    ),
)
table_info(table)
# pa.compute.unique(table["poe_seq"])

## d_labitems

In [ ]:
table = get_table(
    "mimic-iv-2.2/hosp/d_labitems.csv.gz",
    convert_options=pa.csv.ConvertOptions(
        column_types=column_types["d_labitems"],
        strings_can_be_null=True,
        null_values=NULL_VALUES,
        true_values=TRUE_VALUES,
        false_values=FALSE_VALUES,
    ),
)
null_cols = ["label"]
table_info(table)
# pa.compute.unique(table["poe_seq"])

## diagnoses_icd

In [ ]:
table = get_table(
    "mimic-iv-2.2/hosp/diagnoses_icd.csv.gz",
    convert_options=pa.csv.ConvertOptions(
        column_types=column_types["diagnoses_icd"],
        strings_can_be_null=True,
        null_values=NULL_VALUES,
        true_values=TRUE_VALUES,
        false_values=FALSE_VALUES,
    ),
)
table_info(table)
# sorted(pa.compute.unique(table["los"]).drop_null().to_pylist())

## drgcodes

In [ ]:
table = get_table(
    "mimic-iv-2.2/hosp/drgcodes.csv.gz",
    convert_options=pa.csv.ConvertOptions(
        column_types=column_types["drgcodes"],
        strings_can_be_null=True,
        null_values=NULL_VALUES,
        true_values=TRUE_VALUES,
        false_values=FALSE_VALUES,
    ),
)
table_info(table)
# sorted(pa.compute.unique(table["los"]).drop_null().to_pylist())

## emar

In [ ]:
table = get_table(
    "mimic-iv-2.2/hosp/emar.csv.gz",
    convert_options=pa.csv.ConvertOptions(
        column_types=column_types["emar"],
        strings_can_be_null=True,
        null_values=NULL_VALUES,
        true_values=TRUE_VALUES,
        false_values=FALSE_VALUES,
    ),
)
null_cols = ["hadm_id", "scheduletime", "medication", "event_txt"]
table_info(table)
# sorted(pa.compute.unique(table["medication"]).drop_null().to_pylist())

## emar_detail

In [ ]:
table = get_table(
    "mimic-iv-2.2/hosp/emar_detail.csv.gz",
    convert_options=pa.csv.ConvertOptions(
        column_types=column_types["emar_detail"],
        strings_can_be_null=True,
        null_values=NULL_VALUES,
        true_values=TRUE_VALUES,
        false_values=FALSE_VALUES,
    ),
)
table_info(table)
# sorted(pa.compute.unique(table["los"]).drop_null().to_pylist())

In [ ]:
recast_columns = {
    "dose_due": VALUE_TYPE,
    "dose_given": VALUE_TYPE,
    "product_amount_given": VALUE_TYPE,
    "prior_infusion_rate": VALUE_TYPE,
    "infusion_rate": VALUE_TYPE,
    "infusion_rate_adjustment_amount": VALUE_TYPE,
}

for col in recast_columns:
    array = pa.compute.utf8_trim_whitespace(table[col])

    prior_null = pa.compute.is_null(array)
    post_null = pa.compute.is_null(
        pa.Array.from_pandas(
            pd.to_numeric(
                pd.Series(array, dtype="string[pyarrow]"),
                errors="coerce",
                dtype_backend="pyarrow",
                downcast=pa.float32(),
            )
        )
    )

    # mask the values that were initially null or can be cast to float.
    mask = pa.compute.or_(
        prior_null,
        pa.compute.invert(post_null),
    )
    dropped = 1 - pa.compute.mean(mask).as_py()
    print(f"Dropping {dropped:.4%} of rows: {col} can't be cast to float!")
    table = table.filter(mask)
    table = table.set_column(
        table.column_names.index(col),
        col,
        pa.compute.cast(
            array.filter(mask),
            pa.float32(),
        ),
    )
table_info(table)

## hcpcsevents

In [ ]:
table = get_table(
    "mimic-iv-2.2/hosp/hcpcsevents.csv.gz",
    convert_options=pa.csv.ConvertOptions(
        column_types=column_types["hcpcsevents"],
        strings_can_be_null=True,
        null_values=NULL_VALUES,
        true_values=TRUE_VALUES,
        false_values=FALSE_VALUES,
    ),
)
table_info(table)
# sorted(pa.compute.unique(table["los"]).drop_null().to_pylist())

## Labevents

In [ ]:
table = get_table(
    "mimic-iv-2.2/hosp/labevents.csv.gz",
    convert_options=pa.csv.ConvertOptions(
        column_types=column_types["labevents"],
        strings_can_be_null=True,
        null_values=NULL_VALUES,
        true_values=TRUE_VALUES,
        false_values=FALSE_VALUES,
    ),
)
null_cols = ["storetime"]
table_info(table)
# sorted(pa.compute.unique(table["los"]).drop_null().to_pylist())

### drop everything that has no hadm_id or null value or null valueuom

In [ ]:
adm_null = pa.compute.is_null(table["hadm_id"])
val_null = pa.compute.is_null(table["value"])
num_null = pa.compute.is_null(table["valuenum"])
uom_null = pa.compute.is_null(table["valueuom"])
mask = pa.compute.invert(
    pa.compute.or_(
        pa.compute.or_(adm_null, val_null),
        pa.compute.or_(num_null, uom_null),
    )
)
table = pa.compute.filter(table, mask)

# cast value to float
table = table.set_column(
    table.column_names.index("value"),
    "value",
    pa.compute.cast(table.column("value"), pa.float32()),
)
assert pa.compute.all(pa.compute.equal(table["value"], table["valuenum"])).as_py()
table_info(table)

## microbiologyevents

In [ ]:
table = get_table(
    "mimic-iv-2.2/hosp/microbiologyevents.csv.gz",
    convert_options=pa.csv.ConvertOptions(
        column_types=column_types["microbiologyevents"],
        strings_can_be_null=True,
        null_values=NULL_VALUES,
        true_values=TRUE_VALUES,
        false_values=FALSE_VALUES,
    ),
)
null_cols = ["storedate", "storetime", "spec_type_desc"]
table_info(table)
# sorted(pa.compute.unique(table["los"]).drop_null().to_pylist())

### Check dilution

In [ ]:
dilution = table["dilution_text"]
dilution = pa.compute.replace_substring(dilution, "<", "")
dilution = pa.compute.replace_substring(dilution, ">", "")
dilution = pa.compute.replace_substring(dilution, "=", "")
dilution = pa.compute.cast(dilution, pa.float32())
assert (
    pa.compute.all(
        pa.compute.equal(dilution, table["dilution_value"]).drop_null()
    ).as_py()
    is True
)

## omr

In [ ]:
table = get_table(
    "mimic-iv-2.2/hosp/omr.csv.gz",
    convert_options=pa.csv.ConvertOptions(
        column_types=column_types["omr"],
        strings_can_be_null=True,
        null_values=NULL_VALUES,
        true_values=TRUE_VALUES,
        false_values=FALSE_VALUES,
    ),
)
table_info(table)
# sorted(pa.compute.unique(table["los"]).drop_null().to_pylist())

### split results_value (two values stored for blood pressure)

In [ ]:
table = table.set_column(
    table.column_names.index("result_value"),
    "result_value",
    pa.compute.split_pattern(table["result_value"], "/"),
)

df = table.to_pandas().pivot(
    index=["subject_id", "seq_num", "chartdate"],
    columns="result_name",
    values="result_value",
)

for col in (pbar := tqdm(df.columns)):
    pbar.set_postfix(column=f"{col!r}")
    s = df.pop(col).copy()
    m = s.isna()
    s.loc[m] = [[]] * m.sum()
    columns = (
        [f"{col} (systolic)", f"{col} (diastolic)"]
        if "blood pressure" in col.lower()
        else [col]
    )
    frame = pd.DataFrame(s.to_list(), columns=columns, index=s.index)
    for new_col in frame:
        df[new_col] = pd.to_numeric(
            frame[new_col],
            errors="coerce",
            downcast=pa.float32(),
            dtype_backend="pyarrow",
        ).astype(
            "float[pyarrow]"
        )  # required due to "Weight (Lbs)"
        # contains bad data: 210237760.0, the nearest float is 210237758.0
table = pa.Table.from_pandas(df.dropna(how="all").reset_index())
table_info(table)

## patients

In [ ]:
table = get_table(
    "mimic-iv-2.2/hosp/patients.csv.gz",
    convert_options=pa.csv.ConvertOptions(
        column_types=column_types["patients"],
        strings_can_be_null=True,
        null_values=NULL_VALUES,
        true_values=TRUE_VALUES,
        false_values=FALSE_VALUES,
    ),
)
table_info(table)
# sorted(pa.compute.unique(table["los"]).drop_null().to_pylist())

## pharmacy

In [ ]:
table = get_table(
    "mimic-iv-2.2/hosp/pharmacy.csv.gz",
    convert_options=pa.csv.ConvertOptions(
        column_types=column_types["pharmacy"],
        strings_can_be_null=True,
        null_values=NULL_VALUES,
        true_values=TRUE_VALUES,
        false_values=FALSE_VALUES,
    ),
)
table_info(table)
# sorted(pa.compute.unique(table["los"]).drop_null().to_pylist())

In [ ]:
null_cols = [
    "subject_id",
    "hadm_id",
    "pharmacy_id",
    "poe_id",
    "starttime",
    "stoptime",
    "medication",
    "proc_type",
    "status",
    "entertime",
    "verifiedtime",
    "route",
    "frequency",
    # "disp_sched",
    # "infusion_type",
    # "sliding_scale",
    # "lockout_interval",
    # "basal_rate",
    # "one_hr_max",
    # "doses_per_24_hrs",
    # "duration",
    "duration_interval",
    # "expiration_value",
    "expiration_unit",
    # "expirationdate",
    "dispensation",
    # "fill_quantity",
]
table_info(filter_nulls(table, null_cols))

## poe

In [ ]:
table = get_table(
    "mimic-iv-2.2/hosp/poe.csv.gz",
    convert_options=pa.csv.ConvertOptions(
        column_types=column_types["poe"],
        strings_can_be_null=True,
        null_values=NULL_VALUES,
        true_values=TRUE_VALUES,
        false_values=FALSE_VALUES,
    ),
)
null_cols = ["order_provider_id"]
table_info(table)
# sorted(pa.compute.unique(table["los"]).drop_null().to_pylist())

## poe_detail

In [ ]:
table = get_table(
    "mimic-iv-2.2/hosp/poe_detail.csv.gz",
    convert_options=pa.csv.ConvertOptions(
        column_types=column_types["poe_detail"],
        strings_can_be_null=True,
        null_values=NULL_VALUES,
        true_values=TRUE_VALUES,
        false_values=FALSE_VALUES,
    ),
)
table_info(table)
#

### unstack fields

In [ ]:
unstacked_dtypes = {
    "Admit category": "category",
    "Admit to": "category",
    "Code status": "category",
    "Consult Status": "category",
    "Consult Status Time": "datetime64[ns]",
    "Discharge Planning": "category",
    "Discharge When": "category",
    "Indication": "category",
    "Level of Urgency": "category",
    "Transfer to": "category",
    "Tubes & Drains type": "category",
}

df = (
    table.to_pandas()
    .pivot(
        index=["poe_id", "poe_seq", "subject_id"],
        columns="field_name",
        values="field_value",
    )
    .dropna(how="all")
    .astype(unstacked_dtypes)
    .reset_index()
)
table_info(pa.Table.from_pandas(df))

## prescriptions

In [ ]:
table = get_table(
    "mimic-iv-2.2/hosp/prescriptions.csv.gz",
    convert_options=pa.csv.ConvertOptions(
        column_types=column_types["prescriptions"],
        strings_can_be_null=True,
        null_values=NULL_VALUES,
        true_values=TRUE_VALUES,
        false_values=FALSE_VALUES,
    ),
)
table_info(table)
# sorted(pa.compute.unique(table["los"]).drop_null().to_pylist())

### drop val columns that are not float

In [ ]:
table = table.set_column(
    table.column_names.index("dose_val_rx"),
    "dose_val_rx",
    pa.Array.from_pandas(
        pd.to_numeric(
            pd.Series(table["dose_val_rx"]),
            errors="coerce",
            dtype_backend="pyarrow",
            downcast=pa.float32(),
        )
    ),
)
table = table.set_column(
    table.column_names.index("form_val_disp"),
    "form_val_disp",
    pa.Array.from_pandas(
        pd.to_numeric(
            pd.Series(table["form_val_disp"]),
            errors="coerce",
            dtype_backend="pyarrow",
            downcast=pa.float32(),
        )
    ),
)
table_info(table)

In [ ]:
filter_null_cols = [
    "subject_id",
    "hadm_id",
    "pharmacy_id",
    "poe_id",
    "poe_seq",
    "order_provider_id",
    "starttime",
    "stoptime",
    "drug_type",
    "drug",
    "formulary_drug_cd",
    # "gsn",
    "ndc",
    "prod_strength",
    # "form_rx",
    "dose_val_rx",
    "dose_unit_rx",
    "form_val_disp",
    "form_unit_disp",
    # "doses_per_24_hrs",
    "route",
]
table_info(filter_nulls(table, filter_null_cols))

## procedures_icd

In [ ]:
table = get_table(
    "mimic-iv-2.2/hosp/procedures_icd.csv.gz",
    convert_options=pa.csv.ConvertOptions(
        column_types=column_types["procedures_icd"],
        strings_can_be_null=True,
        null_values=NULL_VALUES,
        true_values=TRUE_VALUES,
        false_values=FALSE_VALUES,
    ),
)
table_info(table)
# sorted(pa.compute.unique(table["los"]).drop_null().to_pylist())

## provider

In [ ]:
table = get_table(
    "mimic-iv-2.2/hosp/provider.csv.gz",
    convert_options=pa.csv.ConvertOptions(
        column_types=column_types["provider"],
        null_values=NULL_VALUES,
        true_values=TRUE_VALUES,
        false_values=FALSE_VALUES,
    ),
)
table_info(table)
# sorted(pa.compute.unique(table["los"]).drop_null().to_pylist())

## services

In [ ]:
table = get_table(
    "mimic-iv-2.2/hosp/services.csv.gz",
    convert_options=pa.csv.ConvertOptions(
        column_types=column_types["services"],
        strings_can_be_null=True,
        null_values=NULL_VALUES,
        true_values=TRUE_VALUES,
        false_values=FALSE_VALUES,
    ),
)
table_info(table)
# sorted(pa.compute.unique(table["los"]).drop_null().to_pylist())

## admissions

In [ ]:
table = get_table(
    "mimic-iv-2.2/hosp/admissions.csv.gz",
    convert_options=pa.csv.ConvertOptions(
        column_types=column_types["admissions"],
        strings_can_be_null=True,
        null_values=NULL_VALUES,
        true_values=TRUE_VALUES,
        false_values=FALSE_VALUES,
    ),
)
filter_null_cols = ["admit_provider_id"]
table_info(table)
# sorted(pa.compute.unique(table["los"]).drop_null().to_pylist())

## transfers

In [ ]:
table = get_table(
    "mimic-iv-2.2/hosp/transfers.csv.gz",
    convert_options=pa.csv.ConvertOptions(
        column_types=column_types["transfers"],
        strings_can_be_null=True,
        null_values=NULL_VALUES,
        true_values=TRUE_VALUES,
        false_values=FALSE_VALUES,
    ),
)
table_info(table)
# sorted(pa.compute.unique(table["los"]).drop_null().to_pylist())

In [ ]:
raise

# ICU tables

## caregiver

In [ ]:
table = get_table(
    "mimic-iv-2.2/icu/caregiver.csv.gz",
    convert_options=pa.csv.ConvertOptions(
        column_types=column_types["caregiver"],
        strings_can_be_null=True,
        null_values=NULL_VALUES,
        true_values=TRUE_VALUES,
        false_values=FALSE_VALUES,
    ),
)
table_info(table)
# sorted(pa.compute.unique(table["los"]).drop_null().to_pylist())

## d_items

In [ ]:
table = get_table(
    "mimic-iv-2.2/icu/d_items.csv.gz",
    convert_options=pa.csv.ConvertOptions(
        column_types=column_types["d_items"],
        strings_can_be_null=True,
        null_values=NULL_VALUES,
        true_values=TRUE_VALUES,
        false_values=FALSE_VALUES,
    ),
)
table_info(table)
# sorted(pa.compute.unique(table["los"]).drop_null().to_pylist())

## icustays

In [ ]:
table = get_table(
    "mimic-iv-2.2/icu/icustays.csv.gz",
    convert_options=pa.csv.ConvertOptions(
        column_types=column_types["icustays"],
        strings_can_be_null=True,
        null_values=NULL_VALUES,
        true_values=TRUE_VALUES,
        false_values=FALSE_VALUES,
    ),
)
table_info(table)
# sorted(pa.compute.unique(table["los"]).drop_null().to_pylist())

## ingredientevents

In [ ]:
table = get_table(
    "mimic-iv-2.2/icu/ingredientevents.csv.gz",
    convert_options=pa.csv.ConvertOptions(
        column_types=column_types["ingredientevents"],
        strings_can_be_null=True,
        null_values=NULL_VALUES,
        true_values=TRUE_VALUES,
        false_values=FALSE_VALUES,
    ),
)
table_info(table)
# sorted(pa.compute.unique(table["los"]).drop_null().to_pylist())

## datetimeevents

In [ ]:
table = get_table(
    "mimic-iv-2.2/icu/datetimeevents.csv.gz",
    convert_options=pa.csv.ConvertOptions(
        column_types=column_types["datetimeevents"],
        strings_can_be_null=True,
        null_values=NULL_VALUES,
        true_values=TRUE_VALUES,
        false_values=FALSE_VALUES,
    ),
)
table_info(table)
# sorted(pa.compute.unique(table["los"]).drop_null().to_pylist())

## Procedureeventes

In [ ]:
table = get_table(
    "mimic-iv-2.2/icu/procedureevents.csv.gz",
    convert_options=pa.csv.ConvertOptions(
        column_types=column_types["procedureevents"],
        strings_can_be_null=True,
        null_values=NULL_VALUES,
        true_values=TRUE_VALUES,
        false_values=FALSE_VALUES,
    ),
)
table = table.set_column(
    table.column_names.index("storetime"),
    "storetime",
    table["storetime"].cast(TIME_TYPE, safe=False),
)
table_info(table)
# sorted(pa.compute.unique(table["los"]).drop_null().to_pylist())

## Inputevents

In [ ]:
table = get_table(
    "mimic-iv-2.2/icu/inputevents.csv.gz",
    convert_options=pa.csv.ConvertOptions(
        column_types=column_types["inputevents"],
        strings_can_be_null=True,
        null_values=NULL_VALUES,
        true_values=TRUE_VALUES,
        false_values=FALSE_VALUES,
    ),
)
table_info(table)
# sorted(pa.compute.unique(table["los"]).drop_null().to_pylist())

## Outputevents

In [ ]:
table = get_table(
    "mimic-iv-2.2/icu/outputevents.csv.gz",
    convert_options=pa.csv.ConvertOptions(
        column_types=column_types["outputevents"],
        strings_can_be_null=True,
        null_values=NULL_VALUES,
        true_values=TRUE_VALUES,
        false_values=FALSE_VALUES,
    ),
)
table_info(table)
# sorted(pa.compute.unique(table["los"]).drop_null().to_pylist())

## Chartevents

In [ ]:
table = get_table(
    "mimic-iv-2.2/icu/chartevents.csv.gz",
    convert_options=pa.csv.ConvertOptions(
        column_types=column_types["chartevents"],
        strings_can_be_null=True,
        null_values=NULL_VALUES,
        true_values=TRUE_VALUES,
        false_values=FALSE_VALUES,
    ),
)
table_info(table)
# sorted(pa.compute.unique(table["los"]).drop_null().to_pylist())

### drop everything that has no valueuom

In [ ]:
table = filter_nulls(table, ["valueuom"])
table = table.set_column(
    table.column_names.index("value"),
    "value",
    pa.compute.cast(table.column("value"), pa.float32()),
)
table_info(table)

In [ ]:
df = pl.from_arrow(table).pivot(index="charttime", columns="valueuom", values="value")

In [ ]:
parquet.write_table(table, DATASET_PATH / "chartevents.parquet")

# with polars

In [ ]:
import polars as pl

with ZipFile(ds.rawdata_paths, "r") as file:
    if not file.namelist() == filelist:
        raise ValueError("The dataset contains unknown files!")


def load_gz_table(archive, fname, **options):
    with archive.open(str(fname), "r") as compressed_file:
        with gzip.open(compressed_file, "r") as file:
            # table = pd.read_csv(file)
            # return table
            return pl.read_csv(file, **options)
            # table = pyarrow.csv.read_csv(file, **options)
            # return table


def load_txt(archive, fname):
    with archive.open(str(fname), "r") as file:
        return str(file.read(), encoding="utf8")


def get_table(fname, **options) -> DataFrame:
    fname = Path(fname)
    with ZipFile(ds.rawdata_paths, "r") as archive:
        match fname.suffix:
            case ".gz":
                return load_gz_table(archive, fname, **options)
            case ".txt":
                return load_txt(archive, fname, **options)
            case _:
                raise ValueError

In [ ]:
dtypes = {
    "subject_id": pl.datatypes.UInt32(),
    "hadm_id": pl.datatypes.UInt32(),
    "stay_id": pl.datatypes.UInt32(),
    "itemid": pl.datatypes.UInt32(),
    "charttime": pl.datatypes.Datetime(),
    "storetime": pl.datatypes.Datetime(),
    "value": pl.datatypes.Utf8(),
    "valuenum": pl.datatypes.Float32(),
    "valueuom": pl.datatypes.Utf8(),
    "warning": pl.datatypes.Boolean(),
}


table = get_table("mimic-iv-1.0/icu/chartevents.csv.gz", dtypes=dtypes)